In [2]:
import json
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import numpy as np
from monty.json import MontyEncoder, MontyDecoder
from monty.serialization import loadfn, dumpfn
import pandas as pd
from itertools import chain
import os
import pprint
from pymatgen.analysis.elasticity.elastic import ElasticTensor
from pymatgen.io.pwscf import PWInput, PWOutput

In [3]:
#Load data from folder called "data"

d_3d=loadfn('../data/jdft_3d-5-23-2018.json',cls=MontyDecoder)
d=loadfn('../data/jdft_2d-5-23-2018.json',cls=MontyDecoder)
#d=loadfn('data/jdft_2d.json',cls=MontyDecoder)
df_metals = pd.read_csv('../data/metals_list.csv') #list of all metals on periodic table

In [61]:
#Miscellaneous Functions

#makes a list of perfect squares less than "number"
def perf_squares(number):
    perfect_squares_list = []
    if number >= 0:
        for i in range(1, int(number ** 0.5 + 1)):
            perfect_squares = i**2
            perfect_squares_list.append(perfect_squares)
    return perfect_squares_list

#checks if a space group has inversion symmetry
def has_inversion(group):
    ranges = ((2, 2), (10, 15), (47, 74), (83, 88), (123, 142), (147, 148), (162, 167), (175,176), (191, 194), (200, 206), (221, 230))
    nums = set(chain(*(range(start, end+1) for start, end in ranges)))
    return (group in nums)

#enables clickable urls in the dataframe
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

#Writes a poscar of a structure given the index "num" 
def poscar_wr(num,d):
    x= d[num]['final_str']
    fileout = Poscar(x)
    filename = d[num]['final_str'].formula.replace(" ", "") + '.vasp'
    fileout.write_file(filename)
    print("Wrote " + filename)
    
#Takes as input the dataframe and a list of elements (e.g. ['Mn','P','O'])
def search_for_element(df, el_list):
        searchfor = el_list
        df1 = df[df['Formula'].str.contains('|'.join(searchfor))]
        dfurl = df1.style.format({'Jarvis_URL': make_clickable, 'MP_URL': make_clickable})
        return dfurl
    
#Counts the number of metals per cell in a structure
def count_metals(d_i):
    comp = Composition(d_i['final_str'].formula.replace(" ", ""))
    comp_dict = comp.get_el_amt_dict()
    num_metals = 0.0
    for i in comp_dict:
        for index, row in df_metals.iterrows():
            if i == df_metals['Symbol'].iloc[index]:
                num_metals += comp_dict[i]
    return num_metals

#Gets Elastic Tensor in array form from json object
def get_et(elast_str=''):
    if elast_str == 'na':
        return 'na'
    else:
        cij=np.empty((6, 6), dtype=float)
        elast=np.array(elast_str.split(','),dtype='float')
        count=0
        for ii in range(6):
            for jj in range(6):
                cij[ii][jj]=elast[count]
                count=count+1
        et=ElasticTensor.from_voigt(cij)
        return et

#Calculates Universal Anisotropy Index from elastic tensor
def get_anisotropy(x):
    if x == 'na':
        return np.NaN
    else:
        try:
            return x.universal_anisotropy
        except:
            return np.NaN

#Returns a maximum in a list of Anisotropy indices for a given mpid
def get_anisotropy_list(dat_3d, mpid):
    anisotropy_list = []
    for i in dat_3d:
        if i['mpid'] == mpid:
            anisotropy_list.append(get_anisotropy(get_et(i['elastic'])))
            #pprint.pprint(i)
        x = np.array(anisotropy_list, dtype=np.float64)    
    return np.nanmax(x)

#Returns a maximum in a list of bulk OP band gaps for a given mpid
def get_bulk_gap_list(dat_3d, mpid):
    bulk_gap_list = []
    for i in dat_3d:
        if i['mpid'] == mpid:
            if i['op_gap'] == 'na':
                bulk_gap_list.append(np.NaN)
            else:
                bulk_gap_list.append(i['op_gap'])
        x = np.array(bulk_gap_list, dtype=np.float64)    
    return np.nanmax(x)

#returns kpoint grid size based on a 12x12x1 (21x21x1) grid for relaxation (Lumen SHG) of MoS2
def get_kpt_gridsize(d,num,relax=True,lumen=False):
    mos2_struct = d[255]['final_str']
    mos2_area = mos2_struct.volume/(mos2_struct.lattice.c)
    
    if relax != lumen: 
        if relax == True:
            kpt_den = mos2_area*144
        elif lumen == True:
            kpt_den = mos2_area*441
    else:
        return "Error"
            
    struct = d[num]['final_str']
    area = struct.volume/(struct.lattice.c)
    kpt_gridsize = 0
    for i, val in enumerate(perf_squares(2000)):
        if area*val < kpt_den:
            kpt_gridsize = int(np.sqrt(val))
        else:
            kpt_gridsize = int(np.sqrt(val))
            break
    return kpt_gridsize

#Generate Quantum Espresso Input File
def generate_pw_input(num, d, calctype = 'vc-relax', path = 'QE_relax_inputs/ONCV/'):

    filname = d[num]['final_str'].formula.replace(" ", "")
    struct = d[num]['final_str']
    kpt_gridsize = get_kpt_gridsize(d,num,relax=True,lumen=False)
    kpts = [kpt_gridsize,kpt_gridsize,1]
    
    species_list = []
    for i in struct.species:
        if str(i) not in species_list:
            species_list.append(str(i))
    species_list
    pseudo = {}
    for el in species_list:
        pseudo[el] = el + '_ONCV_PBE_sr.upf'

    control ={'calculation': calctype,
        'restart_mode':'from_scratch',
        'prefix':'bn',
        'pseudo_dir' : '/home/beachk2/PSEUDO/upf_files/PBE/ONCVPSP-master/sg15/',
        'outdir': '/scratch/beachk2/Jarvis/IPA/' + filname,
        'wf_collect':True,
        'forc_conv_thr':1.0E-4,
        'verbosity':'high'
    }
    system = { 'ecutwfc' : 70,
        'occupations':'smearing',
        'smearing' : 'gaussian',
        'degauss':0.005,
        'force_symmorphic' : True
    }
    electrons = {    'mixing_mode' : 'plain',
        'mixing_beta' : 0.7,
        'conv_thr' :  1.0E-8
    }
    ions = {    'ion_dynamics' : 'bfgs'
    }
    if calctype == 'vc-relax':
        cell = {'cell_dofree' : '2Dxy'}
    kpoints_grid = kpts
    PWInput(struct,pseudo=pseudo,control=control,system=system,electrons=electrons,
            ions=ions, cell=cell, kpoints_grid =kpoints_grid).write_file(path +filname + ".relax.in")
    print("Wrote " + filname + ".relax.in")

In [5]:
#Building Dataset

keylist = ['Formula','OP_Gap', 'Bulk_OP_Gap', 'Bulk_Anisotropy', 'Atoms_per_cell','MBJ_Gap',
           'Mag_mom_per_cell', 'Mag_mom_per_metal', 'Mag_mom_per_area','Final_Energy_per_atom',
           'Exf_Energy_per_area','Space_Group','Has_Inversion','Jarvis_URL','MP_URL']
param_dict ={}
for i in keylist:
    param_dict[i] = {'ison':True, 'paramlist':[]}

#Omit columns from calculation here by setting "ison" to False. Default is True.
param_dict['MBJ_Gap']['ison'] = False
param_dict['MP_URL']['ison'] = True
param_dict['Mag_mom_per_cell']['ison'] = True
param_dict['Mag_mom_per_metal']['ison'] = False
param_dict['Mag_mom_per_area']['ison'] = True
param_dict['Bulk_Anisotropy']['ison'] = True
param_dict['Final_Energy_per_atom']['ison'] =False
param_dict['Atoms_per_cell']['ison'] =False
param_dict['Space_Group']['ison'] =False
param_dict['Bulk_OP_Gap']['ison'] =False    
    
for i in d:
    #useful parameters
    stoichiometry = i['final_str'].formula
    cell_vol = i['final_str'].volume
    area = cell_vol/i['final_str'].lattice.c
    num_atoms = i['final_str'].num_sites
    space_group = i['final_str'].get_space_group_info()[1]
    jarvis_url = str("https://www.ctcms.nist.gov/~knc6/jsmol/")+str(i['jid'])+str(".html")
    mp_url=str("https://materialsproject.org/materials/")+str(i['mpid'])+str('/#')
    magnetic_moment = i['magmom']['magmom_out']
    relaxed_energy = i['fin_en']
    exfoliation_energy = i['exfoliation_en']
    OptB88vdW_band_gap = i['op_gap']
    mBJ_band_gap = i['mbj_gap']
    if param_dict['Mag_mom_per_cell']['ison'] == True: 
        number_of_metals = count_metals(i)
        if number_of_metals != 0.0:
            magnetic_mom_metal = magnetic_moment/number_of_metals
        else:
            magnetic_mom_metal = 0.0
    else:
        magnetic_mom_metal = 'na'
    if param_dict['Bulk_Anisotropy']['ison'] == True:
        anisotropy = get_anisotropy_list(d_3d, i['mpid'])
    else:
        anisotropy = np.NaN
    if param_dict['Bulk_OP_Gap']['ison'] == True:
        bulk_gap = get_bulk_gap_list(d_3d, i['mpid'])
    else:
        bulk_gap = np.NaN
    
    #building lists
    param_dict['Formula']['paramlist'].append(stoichiometry)
    param_dict['Atoms_per_cell']['paramlist'].append(num_atoms)
    param_dict['OP_Gap']['paramlist'].append(OptB88vdW_band_gap)
    param_dict['MBJ_Gap']['paramlist'].append(mBJ_band_gap)
    param_dict['Mag_mom_per_cell']['paramlist'].append(magnetic_moment)
    param_dict['Mag_mom_per_area']['paramlist'].append(magnetic_moment/area) 
    param_dict['Mag_mom_per_metal']['paramlist'].append(magnetic_mom_metal)
    param_dict['Final_Energy_per_atom']['paramlist'].append(relaxed_energy/num_atoms)
    param_dict['Exf_Energy_per_area']['paramlist'].append(exfoliation_energy/area)
    param_dict['Space_Group']['paramlist'].append(space_group)
    param_dict['Has_Inversion']['paramlist'].append(has_inversion(space_group))
    param_dict['Jarvis_URL']['paramlist'].append(jarvis_url)
    param_dict['MP_URL']['paramlist'].append(mp_url)
    param_dict['Bulk_Anisotropy']['paramlist'].append(anisotropy)
    param_dict['Bulk_OP_Gap']['paramlist'].append(bulk_gap)

c:\users\kory\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
c:\users\kory\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: All-NaN slice encountered


In [87]:
#Generating dataframe

param_dict['MBJ_Gap']['ison'] = True
param_dict['MP_URL']['ison'] = True
param_dict['Mag_mom_per_cell']['ison'] = True
param_dict['Mag_mom_per_metal']['ison'] = False
param_dict['Mag_mom_per_area']['ison'] = True
param_dict['Bulk_Anisotropy']['ison'] = True
param_dict['Final_Energy_per_atom']['ison'] =False
param_dict['Atoms_per_cell']['ison'] =False
param_dict['Space_Group']['ison'] =True
param_dict['Bulk_OP_Gap']['ison'] =False    

headers =[]
list_of_lists = []
for i in param_dict:
     if param_dict[i]['ison'] == True:
        headers.append(i)
        list_of_lists.append(param_dict[i]['paramlist'])

df = pd.DataFrame(list_of_lists)
df = df.transpose()
df.columns = headers
df_unfiltered = df.copy()

df = df[~df['OP_Gap'].isin(['na'])] #remove items with no band gap
df = df[df['OP_Gap']>.9] #Only include items with band gap > 1eV
df = df[~df['Has_Inversion'].isin([True])] #remove items with inversion symmetry


#Sort by desired parameter
#df = df.sort_values('OP_Gap',ascending=True)
#df = df.sort_values('Exf_Energy_per_area',ascending=True)
#df = df.sort_values('Final_Energy_per_atom',ascending=True)
#df = df.reindex(df.Mag_mom_per_area.abs().sort_values(ascending=False).index)
df = df.reindex(df.Bulk_Anisotropy.abs().sort_values(ascending=False).index)  #sort by absolute value of Anisotropy
#df = df.sort_values('Mag_mom_per_metal',ascending=False)
#df = df.sort_values('Space_Group',ascending=True)


#create dataframe with clickable urls
if param_dict['MP_URL']['ison'] == True:
    dfurl = df.style.format({'Jarvis_URL': make_clickable, 'MP_URL': make_clickable})
    
print("Number of Materials:",df.shape[0])
dfurl

Number of Materials: 72


,Formula,OP_Gap,Bulk_Anisotropy,MBJ_Gap,Mag_mom_per_cell,Mag_mom_per_area,Exf_Energy_per_area,Space_Group,Has_Inversion,Jarvis_URL,MP_URL
34,Bi1 O2,1.9231,-88.0988,na,0.999983,0.084919,13.6974,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-14459.html,https://materialsproject.org/materials/mvc-15971/#
350,B1 N1,4.3585,73.2175,na,0,0,13.0703,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-688.html,https://materialsproject.org/materials/mp-984/#
406,B1 P1 S4,1.5645,63.354,na,1.56e-05,6.24013e-07,3.29613,16,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6190.html,https://materialsproject.org/materials/mp-27724/#
336,B1 N1,4.4818,-62.9937,5.9168,-1e-07,-1.83221e-08,10.7925,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-60594.html,https://materialsproject.org/materials/mp-7991/#
120,Au1 C1 N1,2.1703,57.5359,na,2.81e-05,1.94879e-06,9.41328,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-20029.html,https://materialsproject.org/materials/mp-29196/#
233,Al1 P1 S4,2.5522,38.4982,na,-1.2e-06,-4.14754e-08,2.9029,16,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6265.html,https://materialsproject.org/materials/mp-27462/#
109,Sb2 N2,2.066,10.6803,na,0,0,6.19199,1,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-28311.html,https://materialsproject.org/materials/mvc-15384/#
104,Sn4 S4,1.5751,8.56235,na,9e-05,2.62958e-06,3.76629,39,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-19586.html,https://materialsproject.org/materials/mp-8781/#
517,C1 N1 Cl1,5.3933,8.38588,na,6.92e-05,3.2228e-06,5.15644,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-19540.html,https://materialsproject.org/materials/mp-27502/#
180,W1 S2,1.3372,6.3381,na,0,0,8.64827,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-658.html,https://materialsproject.org/materials/mp-224/#


In [36]:
dfmag = df[np.abs(df['Mag_mom_per_cell'])>1e-4]
dfmag

,Formula,OP_Gap,Bulk_Anisotropy,Mag_mom_per_cell,Mag_mom_per_area,Exf_Energy_per_area,Space_Group,Has_Inversion,Jarvis_URL,MP_URL
34,Bi1 O2,1.9231,-88.0988,0.999983,0.084919,13.6974,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-1...,https://materialsproject.org/materials/mvc-159...
222,Ga2 S2,2.4492,3.52297,0.000233,2.06032e-05,4.97172,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-5...,https://materialsproject.org/materials/mp-2507/#
55,Zn1 W1 O4,3.1428,3.38273,0.0002038,9.83036e-06,6.76375,3,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-5615...
164,Ga2 Se2,1.8729,3.12503,0.000189,1.51401e-05,4.67966,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-1943/#
56,Zn1 Cl2,4.3192,2.72285,0.0001701,1.26632e-05,5.80053,115,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-2...,https://materialsproject.org/materials/mp-5672...
591,Sc2 H2 O4,3.9076,2.26141,0.000159,1.2101e-05,6.70495,31,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-6251...
198,Hg1 I2,1.7517,1.96025,0.0004636,2.42892e-05,5.72727,115,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-23192/#
147,Sc1 Ag1 P2 Se6,1.6513,1.33554,0.0001908,5.2252e-06,2.06411,149,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-13383/#
461,In2 Se2,1.4688,1.18489,-0.0001304,-9.15201e-06,4.77464,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-22691/#
149,Tm1 Ag1 P2 Se6,1.7897,1.13115,-0.0001318,-3.50651e-06,2.06322,149,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-13385/#


In [88]:
#search_for_element(df_unfiltered, ['Mo'])
poscar_wr(34,d)

Wrote Bi1O2.vasp


In [17]:
#Write dataframe to Excel Sheet
#writer = pd.ExcelWriter('magnetic_2D_1eV.xlsx')
#df.to_excel(writer,'Sheet1')
#writer.save()

In [69]:
generate_pw_input(56,d)

Wrote Zn1Cl2.relax.in


In [82]:
PWOutput('relax.out').get_celldm(1)

8.273761

In [13]:
d[34]['final_str']

Structure Summary
Lattice
    abc : 3.6874707872064141 3.6874712160158474 25.806384000000001
 angles : 90.0 90.0 119.99999615321698
 volume : 303.8890358272962
      A : 3.6874707872064141 0.0 0.0
      B : -1.843735393603207 3.1934438725802305 0.0
      C : 0.0 0.0 25.806384000000001
PeriodicSite: Bi (1.8437, 1.0645, 1.9516) [0.6667, 0.3333, 0.0756]
PeriodicSite: O (-0.0000, 2.1290, 0.8533) [0.3333, 0.6667, 0.0331]
PeriodicSite: O (-0.0000, 2.1290, 3.0498) [0.3333, 0.6667, 0.1182]

In [17]:
import pprint as pp

In [83]:

get_kpt_gridsize(d,310)

11

In [86]:
d[34]['final_str'].lattice.a

3.6874707872064141

In [26]:
for index, row in df.iterrows():
    generate_pw_input(index,d)

350
406
336
120
233
109
104
517
180


In [29]:
d[180]['final_str']

Structure Summary
Lattice
    abc : 3.1911968746783743 3.1911968746783743 30.202401999999999
 angles : 90.0 90.0 120.00001371198975
 volume : 266.36628364157332
      A : 1.5955981066407856 -2.7636577528777626 0.0
      B : 1.5955981066407854 2.7636577528777631 0.0
      C : 0.0 0.0 30.202401999999999
PeriodicSite: W (1.5956, -0.9212, 10.6518) [0.6667, 0.3333, 0.3527]
PeriodicSite: S (1.5956, 0.9212, 9.0770) [0.3333, 0.6667, 0.3005]
PeriodicSite: S (1.5956, 0.9212, 12.2266) [0.3333, 0.6667, 0.4048]